# 现代循环神经网络

循环神经网络在实践中一个常见问题是数值不稳定性，
即矩阵连续乘积可以导致梯度消失或梯度爆炸问题。
应用梯度裁剪等技巧来缓解梯度爆炸问题，
但仍需要通过设计更复杂的序列模型解决梯度消失/爆炸问题。

两个广泛使用的网络是
*门控循环单元*（gated recurrent units，GRU）和
*长短期记忆网络*（long short-term memory，LSTM）。


现代循环神经网络的设计目标：
1. 可能早期观测值对预测所有未来观测值具有非常重要的意义，需要在一个记忆元里存储重要的早期信息。
2. 一些词元没有相关的观测值，需要*跳过*隐状态表示中的此类词元。
3. 序列的各个部分之间存在逻辑中断，需要*重置*内部状态表示。

## 门控循环单元（GRU）

门控循环神经网络很好处理循环神经网络中的梯度消失问题，
使得更好地捕获时间步距离很长的序列上的依赖关系。

- *重置门*（reset gate）捕获序列中的短期依赖关系，
- *更新门*（update gate）捕获序列中的长期依赖关系。
- 重置门打开时，门控循环单元包含基本循环神经网络；更新门打开时，门控循环单元可以跳过子序列。


<img src="assets/gru-3.svg" width="50%" height="50%" align="left"/>

- 重置门和更新门

$$
\begin{aligned}
\mathbf{R}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xr} + \mathbf{H}_{t-1} \mathbf{W}_{hr} + \mathbf{b}_r),\\
\mathbf{Z}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xz} + \mathbf{H}_{t-1} \mathbf{W}_{hz} + \mathbf{b}_z),
\end{aligned}
$$

- 候选隐状态

$$
\tilde{\mathbf{H}}_t = \tanh(\mathbf{X}_t \mathbf{W}_{xh} + \left(\mathbf{R}_t \odot \mathbf{H}_{t-1}\right) \mathbf{W}_{hh} + \mathbf{b}_h)
$$

- 隐状态

$$
\mathbf{H}_t = \mathbf{Z}_t \odot \mathbf{H}_{t-1}  + (1 - \mathbf{Z}_t) \odot \tilde{\mathbf{H}}_t
$$

### 定性分析

- 重置门和更新门使用sigmoid函数转换到区间$(0, 1)$，这样就可以进行凸组合。

- 重置门打开时（$\mathbf{R}_t$接近$1$），退化成基本循环神经网络；
重置门关闭时（$\mathbf{R}_t$接近$0$），候选隐状态是以$\mathbf{X}_t$作为输入的多层感知机的结果，起到隐状态*重置*的效果。

- 更新门打开时（$\mathbf{Z}_t$接近$1$），门控循环单元可以跳过子序列保留旧状态。
更新门关闭时（$\mathbf{Z}_t$接近$0$），新的隐状态$\mathbf{H}_t$就会接近候选隐状态$\tilde{\mathbf{H}}_t$。

- 如果整个子序列的所有时间步的更新门都接近于$1$，
则无论序列的长度如何，在序列起始时间步的旧隐状态都将很容易保留并传递到序列结束。

## 长短期记忆网络（LSTM）

长短期存储器（long short-term memory，LSTM `Hochreiter.Schmidhuber.1997`）
比门控循环单元早诞生了近20年，
也比GRU稍微复杂一些。

* 长短期记忆网络有三种类型的门：输入门、遗忘门和输出门。
* 长短期记忆网络的隐藏层输出包括“隐状态”和“记忆元”。只有隐状态会传递到输出层，而记忆元完全属于内部信息。
* 长短期记忆网络可以缓解梯度消失和梯度爆炸。


<img src="assets/lstm-3.svg" width="50%" height="50%" align="left"/>

- 输入门、遗忘门和输出门

$$
\begin{aligned}
\mathbf{I}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xi} + \mathbf{H}_{t-1} \mathbf{W}_{hi} + \mathbf{b}_i),\\
\mathbf{F}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xf} + \mathbf{H}_{t-1} \mathbf{W}_{hf} + \mathbf{b}_f),\\
\mathbf{O}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xo} + \mathbf{H}_{t-1} \mathbf{W}_{ho} + \mathbf{b}_o),
\end{aligned}
$$

- 候选记忆元

$$
\tilde{\mathbf{C}}_t = \text{tanh}(\mathbf{X}_t \mathbf{W}_{xc} + \mathbf{H}_{t-1} \mathbf{W}_{hc} + \mathbf{b}_c)
$$

- 记忆元

$$
\mathbf{C}_t = \mathbf{F}_t \odot \mathbf{C}_{t-1} + \mathbf{I}_t \odot \tilde{\mathbf{C}}_t
$$

- 隐状态

$$
\mathbf{H}_t = \mathbf{O}_t \odot \tanh(\mathbf{C}_t)
$$


### 定性分析

类似于GRU，LSTM也有机制来控制输入和遗忘（或跳过），相对于GRU增加了记忆元。

- 遗忘门和输入门用来控制更新记忆元。
遗忘门打开（$\mathbf{F}_t$接近$1$）且输入门关闭($\mathbf{I}_t$接近$0$)，过去的记忆元$\mathbf{C}_{t-1}$
将被保存并传递到当前时间步。
引入这种设计是为了缓解梯度消失问题，
并更好地捕获序列中的长距离依赖关系。

- 输出门用来控制记忆元是否更新至隐状态。
输出门打开时（$\mathbf{O}_t$接近$1$），能够有效地将所有记忆信息传递给预测部分。
输出门关闭时（$\mathbf{O}_t$接近$0$），只保留记忆元内的所有信息，而不需要更新隐状态。

## 深度循环神经网络

- 在深度循环神经网络中，隐状态的信息被传递到当前层的下一时间步和下一层的当前时间步。
- 深度循环神经网络需要大量的调参（如学习率和修剪） 来确保合适的收敛，模型的初始化也需要谨慎。

<img src="assets/deep-rnn.svg" width="25%" height="25%" align="left"/>

$$
\begin{aligned}
\mathbf{H}_t^{(l)} &= \phi_l(\mathbf{H}_t^{(l-1)} \mathbf{W}_{xh}^{(l)} + \mathbf{H}_{t-1}^{(l)} \mathbf{W}_{hh}^{(l)}  + \mathbf{b}_h^{(l)}) \\
\mathbf{O}_t &= \mathbf{H}_t^{(L)} \mathbf{W}_{hq} + \mathbf{b}_q
\end{aligned}
$$

## 双向循环神经网络

双向循环神经网络（`Schuster.Paliwal.1997`）主要用于序列编码和给定双向上下文的观测估计。

在双向循环神经网络中，每个时间步的隐状态由当前时间步的前后数据同时决定。
网络的前向传播需要在双向层中进行前向和后向递归，
导致双向循环神经网络的计算速度非常慢，
并且网络的反向传播还依赖于前向传播的结果。
因此，梯度求解将有一个非常长的链。

双向循环神经网络实践中常用于文本分类、命名实体识别和预训练模型。

<img src="assets/birnn.svg" width="30%" height="30%" align="left"/>

$$
\begin{aligned}
\overrightarrow{\mathbf{H}}_t &= \phi(\mathbf{X}_t \mathbf{W}_{xh}^{(f)} + \overrightarrow{\mathbf{H}}_{t-1} \mathbf{W}_{hh}^{(f)}  + \mathbf{b}_h^{(f)}) \\
\overleftarrow{\mathbf{H}}_t &= \phi(\mathbf{X}_t \mathbf{W}_{xh}^{(b)} + \overleftarrow{\mathbf{H}}_{t+1} \mathbf{W}_{hh}^{(b)}  + \mathbf{b}_h^{(b)}) \\
\mathbf{H}_t &= (\overrightarrow{\mathbf{H}}_t, \overleftarrow{\mathbf{H}}_t) \\
\mathbf{O}_t &= \mathbf{H}_t \mathbf{W}_{hq} + \mathbf{b}_q
\end{aligned}
$$


## 循环神经网络的问题

由于序列的长距离依赖性，训练GRU/LSTM的训练成本非常高，尤其是深度循环神经网络和双向循环神经网络更难训练。
后面会介绍更高级的替代模型transformer。